<a href="https://colab.research.google.com/github/rabinam24/NLP/blob/main/EnglishToNepali.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
#Checking if GPU is running or not

!nvidia-smi

Sat Feb 24 08:44:25 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [4]:
#Installing the transformers libaries
!pip install datasets transformers[sentencepiece] sacrebleu -q

In [13]:
#Importing the libaries
import os
import sys
import transformers
import pandas as pd
import tensorflow as tf
from datasets import load_dataset
from transformers import AutoTokenizer
from transformers import TFAutoModelForSeq2SeqLM, DataCollatorForSeq2Seq
from transformers import AdamWeightDecay
from transformers import AutoTokenizer, TFAutoModelForSeq2SeqLM

In [14]:
model_checkpoint="d2niraj555/code-llama-7b-text-to-sql"

In [15]:
raw_datasets= load_dataset("ashokpoudel/English-Nepali-Translation-Instruction-Dataset")

In [16]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 3560496
    })
})

In [29]:
raw_datasets['train'][110000]

{'text': '<s>[INST] Please translate "17. Fellow chartered accountant (FCA): The membership of fellow chartered accountant may be granted to the following members in such manner as prescribed: Those who have obtained the certificate of registered auditor of class ‘A’ pursuant to the Auditors Act, 2031 (1975), at the time of the commencement of this Act;" into Nepali [/INST] Sure! here is the translated text into Nepali: \uf076१७. फेलो चार्टर्ड एकाउन्टेन्ट (एफ.सी.ए.) ः देहायका सदस्यहरुलाई तोकिए बमोजिम फेलो चार्टर्ड एकाउन्टेन्टको सदस्यता दिन सकिनेछ ः– (क) यो दफा प्रारम्भ हुँदाका बखत लेखापरीक्षक सम्बन्धी ऐन, २०३१ बमोजिम “क” वर्गको दर्तावाला लेखापरीक्षक प्रमाणपत्र प्राप्त गरेको व्यक्ति, </s>'}

In [18]:
raw_datasets=pd.DataFrame(raw_datasets)

In [19]:
raw_datasets.head()

,train
0,"{'text': '<s>[INST] Please translate ""It happe..."
1,"{'text': '<s>[INST] Please translate ""it happe..."
2,"{'text': '<s>[INST] Please translate ""David sa..."
3,"{'text': '<s>[INST] Please translate ""David sa..."
4,"{'text': '<s>[INST] Please translate ""David sa..."


In [35]:
from sklearn.model_selection import train_test_split


X = raw_datasets

X_train, X_test = train_test_split(X, test_size=0.001, random_state=42)

X_test.shape


(3561, 1)

In [36]:
X_test

,train
1792238,"{'text': '<s>[INST] Please translate ""उ मर्यो ..."
2914247,"{'text': '<s>[INST] Please translate ""यसरी नै ..."
1743961,"{'text': '<s>[INST] Please translate ""More pro..."
1425956,"{'text': '<s>[INST] Please translate ""The girl..."
3415213,"{'text': '<s>[INST] Please translate ""भारतसँग ..."
...,...
277916,"{'text': '<s>[INST] Please translate ""The conc..."
1897902,"{'text': '<s>[INST] Please translate ""४४क.प्र..."
1632347,"{'text': '<s>[INST] Please translate ""Nepal al..."
922877,"{'text': '<s>[INST] Please translate ""The gunm..."


In [40]:
import json
def preprocess_data(input_data):
    try:
        # Extract translated text in Nepali
        nepali_translation = input_data['text'].split('into Nepali: ')[1].split('</s>')[0].strip()
    except IndexError:
        print(f"Error processing: {input_data}")
        return None
    except Exception as e:
        print(f"Unexpected error processing: {input_data}. Error: {e}")
        return None

    # Create the desired format
    output_data = {'translation': {'en': input_data['text'], 'ne': nepali_translation}}

    return output_data

# Process the entire dataset in batches
preprocessed_dataset = []

for i in range(0, len(X_test['train']), batch_size):
    batch = X_test['train'][i:i + batch_size]
    batch_result = [preprocess_data(instance) for instance in batch if preprocess_data(instance) is not None]
    preprocessed_dataset.extend(batch_result)

# Print the result
print(json.dumps(preprocessed_dataset, ensure_ascii=False, indent=2))


Streaming output truncated to the last 5000 lines.
  },
  {
    "translation": {
      "en": "<s>[INST] Please translate \"That is, as per its requirement, the name 'Japan' proved.\" into Nepali [/INST] Sure! here is the translated text into Nepali: अर्थात् त्यसको आवश्यकतानुसार 'जापान' भन्ने नाम सिद्ध भयो। </s>",
      "ne": "अर्थात् त्यसको आवश्यकतानुसार 'जापान' भन्ने नाम सिद्ध भयो।"
    }
  },
  {
    "translation": {
      "en": "<s>[INST] Please translate \"But no opposition candidates conceded defeat, and Carrio spokesman Matias Mendez said seven parties had filed a complaint alleging that ballots were missing or stolen in Buenos Aires province, the country's most populous.\" into Nepali [/INST] Sure! here is the translated text into Nepali: तर विपक्षी उम्मेदवारहरूले पराजय रूपान्तरण गरेनन् र कार्रियो प्रवक्ता मातीस मेन्डेजले सात पार्टीहरूले बुनोस एयर प्रान्तमा मतदान हराइरहेको वा चोरी भएको आरोप लगाएका थिए। </s>",
      "ne": "तर विपक्षी उम्मेदवारहरूले पराजय रूपान्तरण गरेनन् र कार्रि

In [44]:
preprocessed_dataset=pd.DataFrame(preprocessed_dataset)
preprocessed_dataset.shape

(1771, 1)